In [2]:
import wandb
import composuite
from diffusion.utils import *
from diffusion.elucidated_diffusion import Trainer
from diffusion.train_diffuser import SimpleDiffusionGenerator

gin.parse_config_file("/home/anhquanpham/projects/compositional-rl-synth-data/config/diffusion.gin")

base_data_path = '/home/anhquanpham/projects/compositional-rl-synth-data/data'
base_results_folder = '/home/anhquanpham/projects/compositional-rl-synth-data/results'

In [3]:
dataset_type = 'expert'

robot = 'IIWA'
obj = 'Hollowbox'
obst = 'None'
task = 'PickPlace'

results_folder = os.path.join(base_results_folder, robot + '_' + obj + '_' + obst + '_' + task)

dataset = load_single_composuite_dataset(base_path=base_data_path, 
                                         dataset_type=dataset_type, 
                                         robot=robot, obj=obj, 
                                         obst=obst, task=task)
dataset = transitions_dataset(dataset)
inputs = make_inputs(dataset)
inputs = torch.from_numpy(inputs).float()
dataset = torch.utils.data.TensorDataset(inputs)
print("Reached")

Reached


In [6]:
diffusion = construct_diffusion_model(inputs=inputs)
print("Reached")

Skipping normalization for dimensions [195].
Means: tensor([ 1.3141e-01,  8.3188e-02,  8.4500e-01, -2.9661e-02, -1.5846e-02,
         1.6197e-01,  9.7616e-01,  9.7420e-04, -2.2287e-02,  6.5924e-02,
         2.7858e-02, -2.8779e-02,  3.8102e-02,  3.0736e-02,  0.0000e+00,
         0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,
         0.0000e+00, -4.4509e-01, -3.7437e-02,  7.9064e-01, -2.4509e-01,
         1.1627e-03, -2.7353e-02,  3.5910e-02,  1.0000e-01,  1.8000e-01,
         8.0800e-01,  0.0000e+00,  0.0000e+00,  0.0000e+00,  1.0000e+00,
        -4.6166e-02, -1.2863e-01,  7.5249e-02, -2.4509e-01,  1.1627e-03,
        -2.7353e-02,  3.5910e-02, -3.1406e-02,  9.6812e-02, -3.6999e-02,
         0.0000e+00,  0.0000e+00,  0.0000e+00,  1.0000e+00,  1.0000e+00,
         0.0000e+00,  0.0000e+00,  0.0000e+00,  1.0000e+00,  0.0000e+00,
         0.0000e+00,  0.0000e+00,  1.0000e+00,  0.0000e+00,  0.0000e+00,
         0.0000e+00,  9.2718e-01,  4.8639e-01,  9.9534e-01,  1.3250e-01,

In [8]:
wandb_project = 'offline_rl_diffusion'
wandb_entity = ''
wandb_group = 'diffusion_training'

wandb.init(
    project=wandb_project,
    entity=wandb_entity,
    group=wandb_group,
    name=results_folder.split('/')[-1],
)

wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.
wandb: Currently logged in as: phamanhquanbill (phamanhquanbill-university-of-pennsylvania). Use `wandb login --relogin` to force relogin
/home/anhquanpham/.pyenv/versions/first_3.9.6/lib/python3.9/site-packages/wandb/sdk/lib/ipython.py:84: DeprecationWarning: Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython display
  from IPython.core.display import HTML, display  # type: ignore


In [10]:
trainer = Trainer(diffusion, dataset, results_folder=results_folder)
trainer.train()

Number of trainable parameters: 25887940.
Using batch size: 1024


loss: 0.3490:  11%|█▉               | 11204/100000 [3:08:30<24:54:01,  1.01s/it]

KeyboardInterrupt



In [ ]:
env = composuite.make(robot, obj, obst, task, use_task_id_obs=True, ignore_done=False)

In [ ]:
generator = SimpleDiffusionGenerator(env=env, ema_model=trainer.ema.ema_model)
observations, actions, rewards, next_observations, terminals = generator.sample(num_samples=100000)

In [ ]:
np.savez_compressed(
    os.path.join(results_folder, 'samples.npz'),
    observations=observations,
    actions=actions,
    rewards=rewards,
    next_observations=next_observations,
    terminals=terminals
)